In [1]:
import requests
import pandas as pd
import eurostat

url=f"https://appsso.eurostat.ec.europa.eu/nui/print.do"

In [2]:
df = eurostat.get_data_df('lfst_r_lfe2ehour')
df

,unit,age,sex,geo\time,2019,2018,2017,2016,2015,2014,...,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999
0,HR,Y15-24,F,AT,32.0,32.3,31.5,32.0,32.5,32.4,...,33.9,34.2,35.4,35.0,36.9,36.9,36.7,36.6,36.9,36.8
1,HR,Y15-24,F,AT1,29.8,30.3,29.2,29.4,30.8,30.1,...,32.2,32.7,34.1,33.6,35.6,36.5,36.9,36.4,36.1,36.0
2,HR,Y15-24,F,AT11,32.0,33.6,33.2,32.3,35.2,33.8,...,34.5,36.0,34.9,34.4,35.1,37.4,37.3,37.8,38.9,35.7
3,HR,Y15-24,F,AT12,31.5,32.4,32.4,32.1,32.7,32.6,...,33.5,33.3,35.1,35.9,36.9,36.8,37.2,37.7,36.7,36.5
4,HR,Y15-24,F,AT13,27.9,28.0,25.6,26.5,28.5,27.0,...,30.5,31.6,32.8,30.8,34.2,35.8,36.5,34.4,34.8,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,HR,Y_GE15,T,UKM7,36.5,35.8,36.0,36.0,36.4,36.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10496,HR,Y_GE15,T,UKM8,36.1,36.2,35.7,35.6,36.3,36.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10497,HR,Y_GE15,T,UKM9,36.0,36.6,36.9,36.3,36.3,35.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10498,HR,Y_GE15,T,UKN,36.8,36.5,36.5,36.3,36.6,36.7,...,37.3,37.5,37.7,37.8,38.2,37.6,38.1,37.9,38.3,38.7


In [3]:
# Drop all years before 2012 and columns unit and age (same for all entries)
df.columns = df.columns.astype(str)
df = df.drop(df.loc[:, '2011': ].columns, axis = 1)
# Rename to avoid problems using \
df = df.rename(columns={'geo\\time': ' NUTS 2'})

# Extract the data for both sexes (Total) and age group 15-74
df = df[df.sex == 'T']
df = df.drop('sex', axis = 1)
df = df[df.age == 'Y15-74']

In [4]:
# Merge on all entries which are also in the target variable cities to extract only the interesting cities
target_cities = pd.read_csv("Cities_with_codes.csv")
avg_hours = pd.merge(target_cities, df, on=[' NUTS 2'])
avg_hours

,City,City Code,NUTS 2,Country,unit,age,2019,2018,2017,2016,2015,2014,2013,2012
0,Amsterdam,NL002C1,NL32,NL,HR,Y15-74,30.8,30.9,30.8,30.7,30.3,30.4,30.2,30.6
1,Ankara,TR001C1,TR51,TR,HR,Y15-74,44.0,44.7,44.9,45.7,46.8,47.0,48.1,48.7
2,Antwerp,BE002C1,BE2,BE,HR,Y15-74,37.6,37.6,38.1,37.3,37.4,37.4,37.7,37.2
3,Athens,EL001C1,EL30,EL,HR,Y15-74,40.2,40.6,40.5,40.6,40.3,40.3,40.6,40.6
4,Barcelona,ES002C1,ES51,ES,HR,Y15-74,37.2,37.7,37.4,37.5,37.7,37.6,37.6,37.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Vilnius,LT001C1,LT01,LT,HR,Y15-74,38.8,38.8,38.9,38.7,38.8,38.6,38.7,NaN
77,Warsaw,PL001C1,PL91,PL,HR,Y15-74,39.8,39.8,40.2,40.4,40.3,40.2,40.3,NaN
78,Wroclaw,PL004C1,PL51,PL,HR,Y15-74,40.2,40.2,40.3,40.3,40.7,40.8,40.6,40.8
79,Zagreb,HR001C1,HR04,HR,HR,Y15-74,39.8,39.9,39.8,39.4,39.5,39.7,39.6,39.8


In [5]:
# Check for missing values
print('Missing values for columns:')
def NaN_percent(df, column_name):
    row_count = df[column_name].shape[0]
    empty_values = row_count - df[column_name].count()
    return (100.0*empty_values)/row_count
for i in list(avg_hours):
    print("%s: %.2f%%" % (i, NaN_percent(avg_hours,i)))


Missing values for columns:
City: 0.00%
 City Code: 0.00%
 NUTS 2: 0.00%
 Country: 0.00%
unit: 0.00%
age: 0.00%
2019: 1.23%
2018: 1.23%
2017: 1.23%
2016: 1.23%
2015: 1.23%
2014: 1.23%
2013: 1.23%
2012: 7.41%


In [6]:
has_nan = avg_hours[avg_hours.isna().any(axis=1)]
has_nan

,City,City Code,NUTS 2,Country,unit,age,2019,2018,2017,2016,2015,2014,2013,2012
7,Belgrade,-,RS11,RS,HR,Y15-74,42.1,42.3,42.3,42.5,42.7,41.9,42.0,NaN
18,Budapest,HU001C1,HU11,HU,HR,Y15-74,39.2,39.4,39.8,39.9,40.4,40.3,39.7,NaN
28,Edinburgh,UK007C1,UKM7,UK,HR,Y15-74,36.5,35.9,36.1,36.0,36.5,36.5,36.8,NaN
34,Glasgow,UK004C1,UKM3,UK,HR,Y15-74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.9
40,Kaunas,LT002C1,LT02,LT,HR,Y15-74,38.8,38.7,38.3,38.4,38.2,37.8,37.9,NaN
76,Vilnius,LT001C1,LT01,LT,HR,Y15-74,38.8,38.8,38.9,38.7,38.8,38.6,38.7,NaN
77,Warsaw,PL001C1,PL91,PL,HR,Y15-74,39.8,39.8,40.2,40.4,40.3,40.2,40.3,NaN


In [7]:
Glasgow = avg_hours.loc[avg_hours['City'] == 'Glasgow']
# Whole region of Scotland is UKM
# Glasgow is 40% of the population it should be a reasonable approximation
temp  = df[df[' NUTS 2'] == 'UKM']
temp = temp.loc[:,'2019':'2013']
Glasgow.loc[:, '2019':'2013'] = temp.loc[:, '2019':'2013'].to_numpy()
avg_hours[avg_hours['City'] == 'Glasgow'] = Glasgow

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [8]:
# Impute the rest using padding
avg_hours = avg_hours.interpolate(method='pad')
print('Dataframe has NaN values: ', avg_hours.isnull().values.any())
avg_hours

Dataframe has NaN values:  False


,City,City Code,NUTS 2,Country,unit,age,2019,2018,2017,2016,2015,2014,2013,2012
0,Amsterdam,NL002C1,NL32,NL,HR,Y15-74,30.8,30.9,30.8,30.7,30.3,30.4,30.2,30.6
1,Ankara,TR001C1,TR51,TR,HR,Y15-74,44.0,44.7,44.9,45.7,46.8,47.0,48.1,48.7
2,Antwerp,BE002C1,BE2,BE,HR,Y15-74,37.6,37.6,38.1,37.3,37.4,37.4,37.7,37.2
3,Athens,EL001C1,EL30,EL,HR,Y15-74,40.2,40.6,40.5,40.6,40.3,40.3,40.6,40.6
4,Barcelona,ES002C1,ES51,ES,HR,Y15-74,37.2,37.7,37.4,37.5,37.7,37.6,37.6,37.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,Vilnius,LT001C1,LT01,LT,HR,Y15-74,38.8,38.8,38.9,38.7,38.8,38.6,38.7,37.5
77,Warsaw,PL001C1,PL91,PL,HR,Y15-74,39.8,39.8,40.2,40.4,40.3,40.2,40.3,37.5
78,Wroclaw,PL004C1,PL51,PL,HR,Y15-74,40.2,40.2,40.3,40.3,40.7,40.8,40.6,40.8
79,Zagreb,HR001C1,HR04,HR,HR,Y15-74,39.8,39.9,39.8,39.4,39.5,39.7,39.6,39.8


In [9]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
avg_hours

,City,City Code,NUTS 2,Country,unit,age,2019,2018,2017,2016,2015,2014,2013,2012
0,Amsterdam,NL002C1,NL32,NL,HR,Y15-74,30.8,30.9,30.8,30.7,30.3,30.4,30.2,30.6
1,Ankara,TR001C1,TR51,TR,HR,Y15-74,44.0,44.7,44.9,45.7,46.8,47.0,48.1,48.7
2,Antwerp,BE002C1,BE2,BE,HR,Y15-74,37.6,37.6,38.1,37.3,37.4,37.4,37.7,37.2
3,Athens,EL001C1,EL30,EL,HR,Y15-74,40.2,40.6,40.5,40.6,40.3,40.3,40.6,40.6
4,Barcelona,ES002C1,ES51,ES,HR,Y15-74,37.2,37.7,37.4,37.5,37.7,37.6,37.6,37.8
5,Valencia,ES003C1,ES51,ES,HR,Y15-74,37.2,37.7,37.4,37.5,37.7,37.6,37.6,37.8
6,Belfast,UK012C1,UKN0,UK,HR,Y15-74,36.9,36.5,36.5,36.3,36.6,36.7,36.3,36.2
7,Belgrade,-,RS11,RS,HR,Y15-74,42.1,42.3,42.3,42.5,42.7,41.9,42.0,36.2
8,Bergen,NO002C1,NO05,NO,HR,Y15-74,33.7,33.4,33.5,33.5,33.8,34.1,33.7,33.8
9,Berlin,DE001C1,DE30,DE,HR,Y15-74,35.4,35.5,35.4,35.7,35.5,35.7,35.6,35.8


In [10]:
#To be able to merge with the other data
#Transform the columns of each year to a variable year
yearly_data = dict()
avg_working_hours  = pd.DataFrame()
for year in range(2012,2020):
    yearly_data= avg_hours[ list(avg_hours.loc[:,'City':' Country']) + [f"{year}"]]
    yearly_data.insert(4, "Year", year)
    yearly_data = yearly_data.rename(columns={f"{year}": "Unemployment_Rate"})
    avg_working_hours = avg_working_hours.append(yearly_data)
avg_working_hours = avg_working_hours.reset_index(drop=True)

In [11]:
avg_working_hours.to_csv(path_or_buf='Average_Working_Hours.csv')